In [2]:
import json
import csv
import os
import glob
import pandas as pd

In [3]:
with open("../visData.json") as f:
    data = json.load(f)

In [4]:
file_names =  []
for item in data:
    file_names.append('..' + item['Highlight Data'])

In [5]:
combined_csv = pd.concat([pd.read_csv(f) for f in file_names ])

combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [6]:
json_data = pd.read_json('visData.json')
#renames and dropped unwanted columns
json_data = json_data.rename(columns = {'article_sha256': 'Article ID'}).drop(columns = ['Highlight Data','Plain Text','Visualization Link','Article Link', 'ID','articleHash'])
json_data

,Author,Date,Title,Article ID
0,PressTV,2020-03-17 18:56:55,US might be complementing Iran sanctions with ...,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...
1,Thomas Williams,2020-03-11 18:56:55,"Over 1,000 People Cured of Coronavirus in Italy",7d1747829a65cd4dad0faf30a704875a125e9decce3b3b...
2,Kerry Grens,2020-03-12 18:56:55,SARS-CoV-2 Can Live on Plastic and Steel for 2...,be0b18a87d4370fa579180ef26dcb7080598f27f9ec761...
3,Lyubov Stepushova,2020-03-17 18:56:55,2005 CIA Report on Coronavirus Pandemic Discov...,47990959103662e94e796d979018922afddc880fb4b867...
4,Straits Times,2020-03-17 18:56:55,US military may have brought coronavirus to Wu...,3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4...


In [7]:
combined = pd.read_csv('combined_csv.csv')

In [8]:
combined.head(3)

,Article ID,Credibility Indicator ID,Credibility Indicator Category,Credibility Indicator Name,Points,Indices of Label in Article,Start,End,target_text
0,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E0,Evidence,Alternative explanation,-1.868043,[],-1.0,-1.0,NaN
1,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E1,Evidence,Open to evidence,-1.657269,[],-1.0,-1.0,NaN
2,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H0,Holistic,Vaguely Sourced,-1.000000,[],-1.0,-1.0,NaN


In [9]:
combined['Start, End'] = combined[['Start','End']].apply(tuple,axis = 1)

In [10]:
#merge both csv and json dataframes together
full_data = pd.merge(left = combined, right = json_data, on = 'Article ID')
#whats Target Text?
full_data.head()

,Article ID,Credibility Indicator ID,Credibility Indicator Category,Credibility Indicator Name,Points,Indices of Label in Article,Start,End,target_text,"Start, End",Author,Date,Title
0,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E0,Evidence,Alternative explanation,-1.868043,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",PressTV,2020-03-17 18:56:55,US might be complementing Iran sanctions with ...
1,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E1,Evidence,Open to evidence,-1.657269,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",PressTV,2020-03-17 18:56:55,US might be complementing Iran sanctions with ...
2,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H0,Holistic,Vaguely Sourced,-1.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",PressTV,2020-03-17 18:56:55,US might be complementing Iran sanctions with ...
3,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H1,Holistic,Vaguely Sourced,-1.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",PressTV,2020-03-17 18:56:55,US might be complementing Iran sanctions with ...
4,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H2,Holistic,Vague Sourcing,-2.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",PressTV,2020-03-17 18:56:55,US might be complementing Iran sanctions with ...


In [12]:
#Changes Date from DateType object to string type
full_data.Date = full_data.Date.astype(str)
data = full_data.groupby(['Article ID', 'Credibility Indicator Category','Credibility Indicator Name','Credibility Indicator ID','Start, End']).agg(lambda x:list(x)[0])




In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 118 entries, (3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4ec66b45681c86c9301, Evidence, Open to evidence, E0, (-1.0, -1.0)) to (be0b18a87d4370fa579180ef26dcb7080598f27f9ec76181f2cfd851f320da06, Probability, Open to evidence, P1, (2274.0, 2496.0))
Data columns (total 8 columns):
Points                         118 non-null float64
Indices of Label in Article    118 non-null object
Start                          118 non-null float64
End                            118 non-null float64
target_text                    36 non-null object
Author                         118 non-null object
Date                           118 non-null object
Title                          118 non-null object
dtypes: float64(3), object(5)
memory usage: 9.3+ KB


In [14]:
data['Date'] = pd.to_datetime(data['Date'])
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 118 entries, (3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4ec66b45681c86c9301, Evidence, Open to evidence, E0, (-1.0, -1.0)) to (be0b18a87d4370fa579180ef26dcb7080598f27f9ec76181f2cfd851f320da06, Probability, Open to evidence, P1, (2274.0, 2496.0))
Data columns (total 8 columns):
Points                         118 non-null float64
Indices of Label in Article    118 non-null object
Start                          118 non-null float64
End                            118 non-null float64
target_text                    36 non-null object
Author                         118 non-null object
Date                           118 non-null datetime64[ns]
Title                          118 non-null object
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 9.3+ KB


In [15]:
data

Points  \
Article ID                                         Credibility Indicator Category Credibility Indicator Name      Credibility Indicator ID Start, End                   
3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4e... Evidence                       Open to evidence                E0                       (-1.0, -1.0)     -1.000000   
                                                   Holistic                       Qualified Source                H1                       (-1.0, -1.0)      2.000000   
                                                                                                                  H2                       (-1.0, -1.0)      1.000000   
                                                                                                                  H3                       (-1.0, -1.0)      2.000000   
                                                                                                                  H4                       (-1.0, -1.0)      2.000000   
                                                                                  Vaguely Sourced                 H0                       (-1.0, -1.0)     -1.000000   
                                                   Language                       Metaphor unhelpful              L0                       (387.0, 398.0)   -1.000000   
                                                   Probability                    Acknowledges uncertainty        P2                       (2923.0, 3012.0)  0.225094   
                                                                                                                                           (3020.0, 3032.0)  0.000000   
                                                                                                                  P3                       (259.0, 268.0)    0.297684   
                                                                                                                                           (1120.0, 1127.0)  0.000000   
                                                                                  Doesn’t acknowledge uncertainty P4                       (2883.0, 3012.0) -0.565197   
                                                                                                                                           (3095.0, 3099.0)  0.000000   
                                                                                  Inappropriate confidence        P0                       (2834.0, 2919.0) -0.785381   
                                                                                                                                           (3017.0, 3197.0)  0.000000   
                                                                                                                  P1                       (3092.0, 3197.0) -0.678072   
                                                                                  Open to evidence                P5                       (1172.0, 1208.0)  0.430624   
                                                                                                                                           (1299.0, 1408.0)  0.000000   
                                                   Reasoning                      Appeal to Ignorance             R5                       (-1.0, -1.0)     -3.531968   
                                                                                  Begging the Question            R2                       (-1.0, -1.0)     -1.309423   
                                                                                  Equivocation                    R4                       (-1.0, -1.0)     -2.500000   
                                                                                  False Dilemma                   R3                       (-1.0, -1.0)     -2.500000   
                                                                                  Hindsight Bias       